#데이터 확인

In [3]:
import pandas as pd
import numpy as np

In [5]:
data=pd.read_csv("/content/food_orders_new_delhi (1) (2).csv")
data.head()

,Order ID,Customer ID,Restaurant ID,Order Date and Time,Delivery Date and Time,Order Value,Delivery Fee,Payment Method,Discounts and Offers,Commission Fee,Payment Processing Fee,Refunds/Chargebacks
0,1,C8270,R2924,2024-02-01 01:11:52,2024-02-01 02:39:52,1914,0,Credit Card,5% on App,150,47,0
1,2,C1860,R2054,2024-02-02 22:11:04,2024-02-02 22:46:04,986,40,Digital Wallet,10%,198,23,0
2,3,C6390,R2870,2024-01-31 05:54:35,2024-01-31 06:52:35,937,30,Cash on Delivery,15% New User,195,45,0
3,4,C6191,R2642,2024-01-16 22:52:49,2024-01-16 23:38:49,1463,50,Cash on Delivery,NaN,146,27,0
4,5,C6734,R2799,2024-01-29 01:19:30,2024-01-29 02:48:30,1992,30,Cash on Delivery,50 off Promo,130,50,0


#데이터 전처리

In [6]:
#discount 열 숫자만 뽑기 -> 배달앱 수익이면 필요없음
def extract_number(df):
  discount_str = df['Discounts and Offers']

  if not isinstance(discount_str, str): #nan 값 처리
        return 0

  if '%' in discount_str:
    percent = float(discount_str.split('%')[0])
    return df['Order Value'] * (percent/100)

  else:
      percent = float(discount_str.split(' ')[0])
      return df['Order Value'] * (percent/100)
data['Discount Amount'] = data.apply(extract_number, axis=1)

#-----------------------배달앱 전체 수익 계산 ------------------------------------------------------

# 대표 가격대
korean_prices = np.array([1900, 2300, 2700, 3100, 3900])

def price_modifying(col_values):
  col_min = col_values.min()
  col_max = col_values.max()

  bins = np.linspace(col_min,col_max,6)

  def mapper(value):
    idx = np.digitize(value, bins) - 1#구간화 한거 인덱스로 반영
    if idx < 0:
      idx = 0 #최소 조정
    elif idx >= len(korean_prices):
      idx = len(korean_prices) -1 #최대조정
    return korean_prices[idx]
  return col_values.apply(mapper)

columns_to_convert = ['Delivery Fee', 'Commission Fee', 'Payment Processing Fee']

for col in columns_to_convert:
    data[col] =  price_modifying(data[col])

# data['App revenue'] = data['Delivery Fee'] + data['Commission Fee'] + data['Payment Processing Fee']
#----------------------환불된 상태 제거 (수익계산 어려움)---------------------------------------
cond = data['Refunds/Chargebacks'] ==0
data = data.loc[cond]
#-----------------------column 정리 ----------------------------------------------------------------

data = data.drop(columns = ['Discount Amount','Refunds/Chargebacks','Order Value','Order ID','Customer ID','Restaurant ID','Delivery Date and Time','Payment Method','Discounts and Offers'])



In [7]:
data['Order Date and Time'] = pd.to_datetime(data['Order Date and Time'])
data['Month'] = data['Order Date and Time'].dt.month
data['Day'] = data['Order Date and Time'].dt.day
data['Hour'] = data['Order Date and Time'].dt.hour
data['Minute'] = data['Order Date and Time'].dt.minute
data['Second'] = data['Order Date and Time'].dt.second

#데이터 증강

In [22]:
import pickle
monthly_orders = {1 : 39460000//12} #1월 주문수 = 배민 월별 주문건수 / 12

# monthly_orders = {1: 1000000}  # 일단 1백만 개 생성

days_in_month = {1 : 31}

hour_distribution = {
    0: 0.02, 1: 0.01, 2: 0.005, 3: 0.005, 4: 0.005,
    5: 0.01, 6: 0.02, 7: 0.04, 8: 0.06, 9: 0.07,
    10: 0.07,
    11: 0.11,  # 높게
    12: 0.13,  # 가장 높게
    13: 0.11,  # 높게
    14: 0.05, 15: 0.05, 16: 0.06,
    17: 0.10,  # 높게
    18: 0.12,  # 가장 높게
    19: 0.10,  # 높게
    20: 0.04, 21: 0.02, 22: 0.01, 23: 0.01
}

total = sum(hour_distribution.values())
for hour in hour_distribution:
    hour_distribution[hour] /= total #1로 맞춰주기

hours = list(hour_distribution.keys())
probabilities = list(hour_distribution.values())
# 수수료 샘플 정의
delivery_fee_samples = [1900, 2300, 2700, 3100, 3900]
commission_fee_samples = [1900, 2300, 2700, 3100, 3900]
payment_fee_samples = [1900, 2300, 2700, 3100, 3900]

# 날짜 관련 설정
days_in_month = {1: 31}

# 데이터 생성
augment_data = []

for month in [1]:
    daily_order_count = monthly_orders[month] // days_in_month[month]
    for day in range(1, days_in_month[month] + 1):
        for _ in range(daily_order_count):
            hour = np.random.choice(hours, p=probabilities)
            minute = np.random.randint(0, 60)
            second = np.random.randint(0, 60)

            delivery_fee = np.random.choice(delivery_fee_samples)
            commission_fee = np.random.choice(commission_fee_samples)
            payment_fee = np.random.choice(payment_fee_samples)

            date_time_str = f"2024-{month:02d}-{day:02d} {hour:02d}:{minute:02d}:{second:02d}"

            augment_data.append({
                'Order Date and Time': date_time_str,
                'Delivery Fee': delivery_fee,
                'Commission Fee': commission_fee,
                'Payment Processing Fee': payment_fee,
                'Month': month,
                'Day': day,
                'Hour': hour,
                'Minute': minute,
                'Second': second
            })

# 바로 피클로 저장
with open("augmented_data.pkl", "wb") as f:
    pickle.dump(augment_data, f)

print("✅ 저장 완료: augmented_data.pkl")


✅ 저장 완료: augmented_data.pkl


In [44]:
aut = pd.read_pickle('/content/augmented_data.pkl')
aut_df = pd.DataFrame(aut)
aut_df


,Order Date and Time,Delivery Fee,Commission Fee,Payment Processing Fee,Month,Day,Hour,Minute,Second
0,2024-01-01 08:17:32,2300,3900,1900,1,1,8,17,32
1,2024-01-01 06:38:27,3100,3900,3100,1,1,6,38,27
2,2024-01-01 03:13:09,3900,3900,3100,1,1,3,13,9
3,2024-01-01 11:16:53,3100,1900,2700,1,1,11,16,53
4,2024-01-01 16:47:05,3100,1900,2700,1,1,16,47,5
...,...,...,...,...,...,...,...,...,...
3288320,2024-01-31 17:01:00,3100,3100,2300,1,31,17,1,0
3288321,2024-01-31 06:16:39,2300,2700,3900,1,31,6,16,39
3288322,2024-01-31 20:40:04,2300,2300,3100,1,31,20,40,4
3288323,2024-01-31 11:16:55,3100,2300,3900,1,31,11,16,55


In [45]:
aut_df['Order Date and Time'] = pd.to_datetime(aut_df['Order Date and Time'])
aut_df['App revenue'] = aut_df['Delivery Fee'] +aut_df['Commission Fee'] + aut_df['Payment Processing Fee']
aut_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3288325 entries, 0 to 3288324
Data columns (total 10 columns):
 #   Column                  Dtype         
---  ------                  -----         
 0   Order Date and Time     datetime64[ns]
 1   Delivery Fee            int64         
 2   Commission Fee          int64         
 3   Payment Processing Fee  int64         
 4   Month                   int64         
 5   Day                     int64         
 6   Hour                    int64         
 7   Minute                  int64         
 8   Second                  int64         
 9   App revenue             int64         
dtypes: datetime64[ns](1), int64(9)
memory usage: 250.9 MB


# 시간대 나누기

In [28]:
aut_df.info() #order date and time 칼럼 데이터 타입 확인(datetime 형태여야함 !!)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3288325 entries, 0 to 3288324
Data columns (total 9 columns):
 #   Column                  Dtype         
---  ------                  -----         
 0   Order Date and Time     datetime64[ns]
 1   Delivery Fee            int64         
 2   Commission Fee          int64         
 3   Payment Processing Fee  int64         
 4   Month                   int64         
 5   Day                     int64         
 6   Hour                    int64         
 7   Minute                  int64         
 8   Second                  int64         
dtypes: datetime64[ns](1), int64(8)
memory usage: 225.8 MB


In [46]:
#새벽 00~08, 일과시간 08-18, 저녁 18-24
def day_split(time):
    hour = time.hour
    if hour < 8:
        return 'morning'
    elif hour < 18:
        return 'afternoon'
    else:
        return 'evening'

aut_df['day'] = aut_df['Order Date and Time'].apply(day_split)


In [47]:
#밑 시뮬레이션 코드 중 timedelta 이용해서 빼려고 시간만 따로 추출
aut_df['Order Time'] =aut_df['Order Date and Time'].dt.floor('H')


<ipython-input-47-db31addd1738>:2: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  aut_df['Order Time'] =aut_df['Order Date and Time'].dt.floor('H')


In [48]:
# df1 = data[data['day']=='morning']
# df2 = data[data['day']=='afternoon']
# df3 = data[data['day']=='evening']
# df= [df1,df2,df3]
df1 = aut_df[aut_df['day']=='morning']
df2 = aut_df[aut_df['day']=='afternoon']
df3 = aut_df[aut_df['day']=='evening']
df= [df1,df2,df3]

# 시뮬레이션 함수

In [49]:
import pandas as pd
import numpy as np
from datetime import timedelta
import random

def simulation(df_list):
    simulation_minutes = 60
    simulation_count = 10000

    for i, df in enumerate(df_list, 1):  #
        start_time = df['Order Time'].min()
        end_time = df['Order Time'].max() - timedelta(minutes=simulation_minutes)

        random_start_times = [
            start_time + timedelta(seconds=random.randint(0, int((end_time - start_time).total_seconds())))
            for _ in range(simulation_count)
        ]

        simulation_losses = []
        for start in random_start_times:
            end = start + timedelta(minutes=simulation_minutes)
            mask = (df['Order Time'] >= start) & (df['Order Time'] < end)
            total_loss = df.loc[mask, 'App revenue'].sum()
            simulation_losses.append(total_loss)

        average_loss = np.mean(simulation_losses)
        loss_per_minute = average_loss / simulation_minutes


        print(f"--- [DataFrame {i}] ---")
        print(f"1시간 장애 시 평균 손해액: {average_loss:,.0f}원")
        print(f"분당 손해액 추정: {loss_per_minute:,.0f}원")




In [50]:
simulation(df)

--- [DataFrame 1] ---
1시간 장애 시 평균 손해액: 3,550,962원
분당 손해액 추정: 59,183원
--- [DataFrame 2] ---
1시간 장애 시 평균 손해액: 24,801,091원
분당 손해액 추정: 413,352원
--- [DataFrame 3] ---
1시간 장애 시 평균 손해액: 9,408,917원
분당 손해액 추정: 156,815원
